In [3]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from zcode.constants import *
import zcode.math as zmath
import zcode.plot as zplot
import zcode.inout as zio

In [4]:
np.seterr(divide='ignore', invalid='ignore');

mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times']})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})

In [16]:
PATH_INPUT_DATA = ("/Users/lzkelley/Research/working/arepo/illustris/redesign/mike_katz/"
                   "data_ready_june_snap_lim_1.txt")

input_data = np.genfromtxt(PATH_INPUT_DATA, names=True, dtype=None)
print("Shape = {}".format(input_data.shape))
input_data.dtype.names

Shape = (18878,)


('merger',
 'snap_prev_in',
 'snapshot_prev_out',
 'snapshot_fin_out',
 'subhalo_prev_in',
 'subhalo_prev_out',
 'subhalo_fin_out',
 'id_new_prev_in',
 'id_new_prev_out',
 'id_new_fin_out',
 'mass_new_prev_in',
 'mass_new_prev_out',
 'mass_new_fin_out',
 'redshift',
 'separation',
 'coordinates_x',
 'coordinates_y',
 'coordinates_z',
 'gamma',
 'vel_disp_prev_in',
 'vel_disp_prev_out',
 'vel_disp_fin_out',
 'stellar_mass_prev_in',
 'stellar_mass_prev_out',
 'stellar_mass_fin_out',
 'total_mass_prev_in',
 'total_mass_prev_out',
 'total_mass_fin_out')

In [23]:
masses = np.array([input_data['mass_new_prev_in'], input_data['mass_new_prev_out']]).T
snaps = np.array([input_data[kk] for kk in ['snap_prev_in', 'snapshot_prev_out', 'snapshot_fin_out']]).T
redz = input_data['redshift']
sepa = input_data['separation']

m2, m1 = masses.T
mt = m1 + m2
mr = m2/m1

print(all(snaps[:, 0] == snaps[:, 1]))

print("Mtot = " + zmath.stats_str(mt, log=True))
print("Mrat = " + zmath.stats_str(mr, log=False))
print("redz = " + zmath.stats_str(redz, log=False))
print("sepa = " + zmath.stats_str(sepa, log=False))


True
Mtot = (5.46, 6.57, 7.45, 8.75, 10.31), for (0%, 16%, 50%, 84%, 100%) (log values)
Mrat = (7.02e-06, 2.83e-03, 9.20e-02, 5.36e-01, 1.00e+00), for (0%, 16%, 50%, 84%, 100%)
redz = (1.11e-04, 4.38e-01, 1.21e+00, 2.42e+00, 7.67e+00), for (0%, 16%, 50%, 84%, 100%)
sepa = (1.90e-01, 1.58e+00, 2.17e+00, 3.19e+00, 1.45e+02), for (0%, 16%, 50%, 84%, 100%)
